In [1]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from scipy.stats import bernoulli
from scipy.integrate import quad

In [114]:
pi_theta = 1/3240.0
varcov_lambda = np.array(([0.25,0,0,0],[0,0.25,0,0],
                          [0,0,1,0], [0,0,0,1]))

In [3]:
#scale parameters
#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)
trois = np.delete(arr=trois, obj=0)
trois = trois[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))

In [115]:
prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 30., size=1000)
prior_delta = np.random.uniform(-30., 30., size=1000)
prior_gen = np.vstack((prior_alpha,prior_beta,prior_gamma, prior_delta))
prior_gen = np.transpose(prior_gen)

In [5]:
def generation(n, alpha=1.7, beta=0.9, gamma=10, delta=10):
    # Initialize samples array with zeros
    sample = np.zeros(n)
    
    # Constants that do not depend on the sample index and thus can be computed once
    if alpha != 1:
        S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
        B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))

    for i in range(n):
        U = np.random.uniform(-np.pi/2, np.pi/2)
        W = -np.log(1 - np.random.uniform(0,1))
        
        # Handle the case alpha = 1 separately
        if alpha != 1:
            part1 = np.sin(alpha * (U + B_alpha_beta)) / (np.cos(U) ** (1 / alpha))
            part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
            sample[i] = S_alpha_beta * part1 * part2
        else:
            sample[i] = (2 / np.pi) * ((np.pi / 2 + beta * U) * np.tan(U) - beta * np.log((np.pi / 2 * W * np.cos(U))/(np.pi+beta*U)))

    # Apply scaling and location shifting
    sample = gamma * sample + delta
    return sample

In [6]:
def gaussian_ker(u=0, y=0, epsilon=1):
    """gaussian kernel for weights
    
    Parameters
    -------------------
    y : float, or array-like
    the point we have, the output

    u : float, or array-like
    the point from which we want to calculate a weight

    epsilon : int, float
    the scale parameter for which we want to compute the kernel
    ----------
    """

    w = (1/np.sqrt(2*np.pi*(epsilon**2)))*np.exp(-(np.abs((u-y)))**2/(2*(epsilon**2)))/epsilon
    return w

In [7]:
def zolotarev_transfo(sample, xi=0.25):
    """function to use for the estimation based on the zolotarev transformation

    Parameters
    --------------------------
    Sample : array-like
    Sample to do the transformation on

    xi : int, float
    The constant used in the transformation
    --------------------------
    """
    if xi<=0 or xi>1/2 :
        raise ValueError('Xi must be between 0 and 1/2')
    taille = len(sample)
    Z = []
    for i in range(int(taille/3)):
        transfo = sample[3*i-2] - xi*sample[3*i-1] - (1 - xi)*sample[3*i]
        Z.append(transfo)
    V = []
    U = []
    for i in range(len(Z)):
        V.append(np.log(np.abs(Z[i])))
        U.append(np.sign(sample[i]))
    V = np.array(V)
    U = np.array(U)
    S_U_squared = (np.std(U))**2
    S_V_squared = (np.std(V))**2
    nu_tilde = (6/(np.pi)**2)*S_V_squared - (3/2)*S_U_squared + 1
    etha_hat = np.mean(U)
    tau_hat = np.mean(V)
    nu_hat = 0
    if nu_tilde > ((1+np.abs(etha_hat))**2)/4:
        nu_hat = nu_tilde
    else:
        nu_hat = ((1+np.abs(etha_hat))**2)/4
    delta_hat = np.mean(sample)
    S_2 = np.array((nu_hat, etha_hat, tau_hat, delta_hat))
    return S_2

In [8]:
default = generation(n=1000)

In [9]:
def custom_integrand(u, epsilon_t):
    return np.exp(-u**2 / (2 * epsilon_t**2)) / (np.sqrt(2 * np.pi) * epsilon_t)

In [118]:
def pi_lf(epsilon_t, sample=default, alpha=1.7, beta=0.9, gamma=10, delta=10, possession=False):
    """function to have the likelihood free density given in the article, the data we create if there is
    no data provided is based on the parameters of 'generation', so for any comparaison between the values of theta,
    those are the parameters to change

    Parameters
    -----------------
    N : int
    number of priors we want to generate
        
    alpha, beta, gamma, delta : int, float
    the parameters with which we want to compute the true data if we don't already have it


    sample : array-like
    the data we observe

    summary_statistic : array-like
    summary statistic used to make tests on the distance between two datasets

    epsilon_t : int, float
    scale parameter, determines 

    Possession : boolean
    by default, set to false, to determine if we want to generate the data or if we already have it

    method : str
    the method with which we would compute the summary statistics
    -------------------
    """
    if possession == False :
        data = generation(n=1000)
    else :
        data = sample
    true_param = zolotarev_transfo(sample=data)
    proposal = generation(n=1000, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    zolo_proposal = zolotarev_transfo(sample=proposal)
    weight = gaussian_ker(u=np.linalg.norm(true_param-zolo_proposal), epsilon=epsilon_t)
    pi_lf = weight*pi_theta


    normalization_constant, _ = quad(custom_integrand, 0, np.inf, args=(epsilon_t,))


    pi_lf = weight / normalization_constant

    return pi_lf
    
    return pi_lf

weight_gen[i] = pi_lf(epsilon_t=periode_t, theta_i, sample=y)/mutation_density(theta_i, t)

In [116]:
def mutation_creation(theta_t_1):
    """a function to sample a theta_t^(i) as in the paper, according to the distribution of the mutation
    kernel

    Parameters
    -------------------------
    theta_t_1 : array,
    the parameter at step t - 1
    --------------------------
    """
    weighted_random = np.zeros((1000,4))
    for i in range(1000):
        weighted_random[i] = weight_gen[i]*multivariate_normal.rvs(mean=theta_t_1, cov=varcov_lambda, size=1)
    mt_theta = np.sum(a=weighted_random, axis=0)
    return mt_theta

In [117]:
def mutation_density(theta_t, theta_t_1):
    """function that evaluates the pdf of the kernel for a given theta

    Parameters
    ----------------------
    theta_t : 1-D array, 
    the parameter of which we will estimate the probability density at step t

    theta_t_1 : array, 
    the parameter at a step before
    ----------------------
    """
    weighted_estimated = np.zeros(1000)
    for i in range(1000):
        weighted_estimated[i] = weight_gen[i]*multivariate_normal.pdf(theta_t, mean=theta_t_1,
                                                                      cov=varcov_lambda)
    mt_theta = np.sum(a=weighted_estimated, axis=0)
    return mt_theta
    

In [57]:
def resample(prior_gen=prior_gen, weight_gen=weight_gen, N=1000):
    """Resample N particles (rows) from 'prior_gen' matrix based on weights in 'weight_gen'.
    
    Parameters
    ------------------------
    prior_gen : 1-D or 2-D array,
    matrix of prior particles where each row is a particle

    weight_gen : 1-D or 2-D array 
    vector of weights corresponding to particles
    
    N : int,
    number of particles to resample.
    -------------------------
    """

    weights = weight_gen / np.sum(weight_gen)
    
    indices = np.random.choice(prior_gen.shape[0], size=N, replace=True, p=weights)
    
    resampled_particles = prior_gen[indices, :]
    
    return resampled_particles

### Initialization

In [161]:
y = generation(n=1000)

In [162]:
weight_gen = np.zeros((100))

In [163]:
prior_alpha = np.random.uniform(1.1, 2., size=100)
prior_beta = np.random.uniform(-1., 1, size=100)
prior_gamma = np.random.uniform(0., 30., size=100)
prior_delta = np.random.uniform(-30., 30., size=100)
prior_gen = np.vstack((prior_alpha,prior_beta,prior_gamma, prior_delta))
prior_gen = np.transpose(prior_gen)

In [69]:
for i in tqdm(range(1000)):
    weight_gen[i] = pi_lf(epsilon_t=scale_param[0], alpha=prior_alpha[i],
                          beta=prior_beta[i], gamma=prior_gamma[i],
                          delta=prior_delta[i], possession=True, sample=y)/pi_theta
    

100%|██████████| 1000/1000 [00:15<00:00, 64.10it/s]


In [158]:
prior_t = prior_gen
prior_t_1 = prior_gen

In [164]:
weight_gen = np.ones(100)*1/100

In [ ]:
for k in tqdm(range(10)):
    c_t = np.quantile(a=weight_gen, q=0.9) #we set the threshold as in the paper, before
    weight_gen = np.ones(1000)*1/1000
    prior_t_1 = prior_t
    for i in tqdm(range(100)):
        p_i = 0
        decision = 0
        compteur = 0
        while decision != 1 and compteur<100:
            theta_proposal = mutation_creation(prior_t_1[i])
            weight_gen[i] = pi_lf(possession=True, sample=y, epsilon_t=scale_param[k],
                                  alpha=theta_proposal[0], beta=theta_proposal[1],
                                  gamma=theta_proposal[2], delta=theta_proposal[2])/mutation_density(theta_t=prior_t[i],
                                                                                                     theta_t_1=prior_t_1[i])
            p_i = np.min((1,weight_gen[i]/c_t))
            if 0 < p_i < 1:
                decision = bernoulli.rvs(p_i, size=1)
                if decision == 1:
                    prior_t[i] = theta_proposal
                    weight_gen[i] = weight_gen[i]/p_i
            compteur += 1
            if compteur == 100:
                weight_gen[i] = 0
        prior_t = resample(prior_gen=prior_t, weight_gen=weight_gen, N=1000)
        

  1%|          | 1/100 [00:23<38:53, 23.57s/it]/tmp/ipykernel_2789/202882002.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  weight_gen[i] = pi_lf(possession=True, sample=y, epsilon_t=scale_param[k],
/tmp/ipykernel_2789/3625153862.py:7: RuntimeWarning: invalid value encountered in tan
  S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
/tmp/ipykernel_2789/3625153862.py:8: RuntimeWarning: invalid value encountered in tan
  B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))
/tmp/ipykernel_2789/3625153862.py:17: RuntimeWarning: invalid value encountered in scalar divide
  part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
/tmp/ipykernel_2789/342708237.py:15: RuntimeWarning: invalid value encountered in scalar multiply
  weighted_estimated[i] = weight_gen[i]*multivariate_normal.pdf(theta_t, mean=theta_t_1,

  2%|▏         | 2/100 [00:46<37:41, 23.08s/it]/tmp/ipykernel_2789/3625153862.py

In [122]:
mutation_density(theta_t=prior_gen[1], theta_t_1=prior_gen[817])

3.084548285042548e-63

In [28]:
np.min((1,2))

1

In [45]:
prior_t = resample(prior_gen=prior_t, weight_gen=weight_gen, N=1000)

In [46]:
np.mean(prior_t-prior_t_1)

nan

In [60]:
prior_t_1

array([[ 1.12279735e+00, -7.86139478e-01,  2.50126877e+02,
         2.07653617e+02],
       [ 1.24053381e+00, -4.52239128e-01,  2.43781030e+02,
         2.72708979e+02],
       [ 1.43582204e+00, -3.40154625e-01,  8.96212668e+01,
        -1.74307205e+02],
       ...,
       [ 1.82467081e+00, -3.87278439e-01,  2.25181286e+02,
        -2.01689040e+02],
       [ 1.51915484e+00,  9.83212805e-01,  1.91245157e+01,
         1.30318940e+02],
       [ 1.23404050e+00, -2.50068271e-01,  1.52248619e+02,
         1.51108207e+02]])

In [79]:
prior_t

array([[  1.96650002,   0.69319941, 144.40500884, 100.86267187],
       [  1.96650002,   0.69319941, 144.40500884, 100.86267187],
       [  1.96650002,   0.69319941, 144.40500884, 100.86267187],
       ...,
       [  1.96650002,   0.69319941, 144.40500884, 100.86267187],
       [  1.96650002,   0.69319941, 144.40500884, 100.86267187],
       [  1.96650002,   0.69319941, 144.40500884, 100.86267187]])

In [104]:
np.count_nonzero(np.isnan(prior_gen))

0

In [103]:
prior_gen.shape

(1000, 4)

In [102]:
pi_lf(possession=True, sample=y, epsilon_t=scale_param[1],alpha=1.7, beta=0.9,gamma=10, delta=10)/mutation_density(theta_t=np.array((1.7,0.9,10,10)),theta_t_1=np.array((1.7,0.9,10,10)))

9.721968825980867e-06

In [111]:
mutation_density(prior_gen[0], prior_gen[15])

0.0

In [105]:
np.count_nonzero(np.isnan(prior_t))

0

In [106]:
np.count_nonzero(np.isnan(prior_t_1))

0